In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import os as io
from PIL import Image as im
import torch.nn as nn
import torch.optim as optim

import torch
import copy
import os 

In [2]:
class PruebaDatastes(Dataset):
    """Classifier dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.metadata = pd.read_csv(csv_file)
        self.metadata =  self.metadata[['filename', 'label']]
        self.root_dir = root_dir
        self.transform = transform

        self.X_train = self.metadata['filename']
        self.y_train  = torch.tensor(self.metadata['label'], dtype=torch.long)
        
    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir,
                                self.X_train[index])
        
        image = im.open(img_name).convert('RGB')
        # reescale
        image = image.resize((224,224))
        
        label = self.y_train[index]

        if self.transform:
            image = self.transform(image)

        return image, label

data_transform = transforms.Compose([
        transforms.ToTensor(),
    
    ])


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [3]:
tranform_datasets = PruebaDatastes(csv_file='/Users/ezapata/Downloads/metadata-ejemplo.csv',
                                 root_dir='/Users/ezapata/Downloads/Ejemplo/', transform=data_transform)

In [4]:
dataloader = DataLoader(tranform_datasets, batch_size=4,
                        shuffle=True, num_workers=1)

In [ ]:
for i_batch , y in dataloader:
    print(i_batch, y)

In [6]:
resnet_model = models.resnet18(pretrained=True)
# freeze
set_parameter_requires_grad(resnet_model, True)
num_features = resnet_model.fc.in_features

In [19]:
num_clases = 4

In [21]:
resnet_model.fc = nn.Linear(num_features, num_clases)
input_size = 224
opt = optim.Adam(resnet_model.parameters(), lr=3e-3)
loss = nn.CrossEntropyLoss()

In [32]:
def train(num_epochs, model, opt, loss_fn, dataloaders):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss = 0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        model.train()
        
        running_loss = 0.0
        running_corrects = 0
         # Iterate over data.
            
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward() # backprop
            opt.step() 
            opt.zero_grad()
            
            _, preds = torch.max(outputs, 1)
            train_loss += loss.item() * len(inputs)

         # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders.dataset)
        
        print('{} Loss: {} Acc: {}'.format('phase', epoch_loss, 'n'))
         



In [33]:
train(2, resnet_model, opt, loss, dataloader)

Epoch 0/1
----------
phase Loss: 0.06781034469604492 Acc: n
Epoch 1/1
----------
phase Loss: 0.09424811999003092 Acc: n
